In [ ]:
%matplotlib notebook
%load_ext autoreload
%autoreload 2

In [ ]:
import cvxpy as cp
import numpy as np
from gcspy import GraphOfConvexSets

In [ ]:
gcs = GraphOfConvexSets()

n = 3
squares = [0, 2, 4, 6, 8]
for i in range(n):
    for j in range(n):
        norm = np.inf if gcs.num_vertices() in squares else 2
        v = gcs.add_vertex(str((i,j)))
        x = v.add_variable(2)
        c = np.array([i, j])
        v.add_constraint(cp.norm(x - c, norm) <= .25)

# Shortest path

In [ ]:
gcs.edges = []
edges = [
    ((0, 0), (1, 0)),
    ((1, 0), (2, 0)),
    ((2, 0), (2, 1)),
    ((0, 1), (0, 0)),
    ((0, 1), (0, 2)),
    ((1, 1), (1, 0)),
    ((1, 1), (0, 1)),
    ((1, 1), (1, 2)),
    ((2, 1), (1, 1)),
    ((2, 1), (2, 2)),
    ((1, 2), (0, 2)),
    ((1, 2), (2, 2)),
        ]
for tail_name, head_name in edges:
    tail = gcs.get_vertex_by_name(str(tail_name))
    head = gcs.get_vertex_by_name(str(head_name))
    edge = gcs.add_edge(tail, head)
    edge.add_cost(cp.norm(head.variables[0] - tail.variables[0], 2))

In [ ]:
source = gcs.vertices[0]
target = gcs.vertices[2]
prob = gcs.solve_shortest_path(source, target)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.gca().set_aspect('equal')
plt.axis('off')
gcs.plot_2d()
gcs.plot_subgraph_2d()
plt.savefig('shortest_path.pdf')

# Traveling salesman

In [ ]:
gcs.edges = []
for i in range(n):
    for j in range(n):
        for k in range(n):
            for l in range(n):
                dx = abs(i - k)
                dy = abs(j - l)
                if dx <= 1 and dy <= 1 and dx + dy != 0:
                    tail = gcs.get_vertex_by_name(str((i, j)))
                    head = gcs.get_vertex_by_name(str((k, l)))
                    edge = gcs.add_edge(tail, head)
                    edge.add_cost(cp.norm(head.variables[0] - tail.variables[0], 2))

In [ ]:
prob = gcs.solve_traveling_salesman()

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.gca().set_aspect('equal')
plt.axis('off')
gcs.plot_2d()
gcs.plot_subgraph_2d()
plt.savefig('traveling_salesman.pdf')

# Facility location

In [ ]:
gcs.edges = []
customers = squares
facilities = [i for i in range(gcs.num_vertices()) if i not in squares]
for facility in facilities:
    for customer in customers:
        tail = gcs.vertices[facility]
        head = gcs.vertices[customer]
        edge = gcs.add_edge(tail, head)
        edge.add_cost(cp.norm(tail.variables[0] - head.variables[0], 2))

In [ ]:
prob = gcs.solve_facility_location(facilities)

In [ ]:
import matplotlib.pyplot as plt
plt.figure()
plt.gca().set_aspect('equal')
plt.axis('off')
gcs.plot_2d()
gcs.plot_subgraph_2d()
plt.savefig('facility_location.pdf')